# WorkShop 2

Let's play with pandas and answer "stakeholders" _questions_.

Problems $2-13$ rely on your `Problem 1` answer.

## Problem  1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of __energy_df__.

Keep in mind that this is an _Excel file_, and not a _comma separated values file_. Also, make sure to _exclude_ the footer and header information from the datafile. The first _two columns_ are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

```python
['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
```

Convert `Energy Supply` to _gigajoules_ (there are $1000000$ _gigajoules_ in a _petajoule_**). For all countries which have missing data (_e.g._ data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (_for use in later questions_):

```bash
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

There are also several countries with parenthesis in their name. Be sure to _remove these_, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with _Numeric digits_ in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a _csv_ containing countries' GDP from $1960$ to $2015$ from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame __gdp_df__. 

Make sure to _skip the header_, and _rename_ the following list of countries:

```bash
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finally, load the [Scimago Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their _journal contributions_ in the aforementioned area. Call this DataFrame __scimago_df__.

Join the _thrgoee datasets_: _energy\_df_, _gdp\_df_ and _scimago\_df_, into a new dataset (using the _intersection_ of _country names_). Use only the _last_ $10$ years ($2006-2015$) of __GDP data__ and only the _top 15 countries_ by __Scimagojr 'Rank'__ (_Rank 1 through 15_). 

The _index_ of this DataFrame should be the __name__ of the country, and the _columns_ should be:
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']`.

_This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank"._

In [126]:
#!pip install xlrd
#!pip install openpyxl
import pandas as pd
import numpy as np
import re

# ---- My created functions

def clean_dataframe(df:pd.DataFrame, iloc:int ,start:int, stop:int, del_Col:list): 
    df.columns = [(dict(df.iloc[iloc])).get(i,i) for i in df.columns]
    df = df.iloc[start:stop]
    df = df.drop(columns=del_Col)
    return df

def transform_country(dictionary:dict, serie):
    serie = [re.compile(r'\(.*?\)').sub('', word).strip() for word in serie]
    serie = [re.compile(r'\d{1,2}$').sub('', word) for word in serie]
    serie = [dictionary.get(country, country) for country in serie]
    serie = sorted(serie)
    return serie

# ---- Problem 1

def generate_energy_df():
    energy_df = pd.read_excel('./assets/Energy Indicators.xls')
    dict1 = {
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"
    }

    energy_df = energy_df.drop(columns=['Unnamed: 0','Unnamed: 1'])
    energy_df = clean_dataframe(energy_df,8, 17,244, [])
    energy_df = energy_df.rename(columns={'Renewable Electricity Production':'% Renewable'})
    energy_df['Energy Supply'] = energy_df['Energy Supply'].apply(lambda x: np.nan if x=='...' else x)*10**6
    energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].apply(lambda x: np.nan if x=='...' else x)
    energy_df['Country'] = transform_country(dict1, energy_df['Country'])
    energy_df = energy_df.set_index('Country')
    return energy_df

def get_GDP_df():
    
    gdp_df = pd.read_csv('assets/world_bank.csv')
    dict_gdp_df = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran","Hong Kong SAR, China": "Hong Kong"}
    gdp_df = clean_dataframe(gdp_df,3, 5,269, ['Indicator Code','Indicator Name'])
    gdp_df = gdp_df.rename(columns={'Country Name':'Country','Country Code':'Code'})    
    gdp_df["Country"] = transform_country(dict_gdp_df, gdp_df['Country'])
    gdp_df = gdp_df.set_index('Country')
    return gdp_df

def get_scimago_df():
    scimago_df = pd.read_excel('./assets/scimagojr-3.xlsx')
    scimago_df = scimago_df.set_index('Country')
    return scimago_df

def answer_one():
    df = ((get_scimago_df()[:15]).join(generate_energy_df(), how='inner')).join(get_GDP_df(), how='inner')
    col_df = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per capita', '% Renewable', 2006, 2007, 2008,2009, 2010, 2011, 2012, 2013, 2014, 2015]
    return df[col_df]

In [127]:
# example of test cases
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should h.ave 20 columns and 15 entries!"
answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per capita,% Renewable,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.75491,1.352564e+09,1.557757e+09,1.661356e+09,1.640250e+09,1.664311e+09,1.730364e+09,1.749085e+09,1.767367e+09,1.816851e+09,1.862692e+09
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.57098,3.606999e+11,3.922741e+11,4.129777e+11,3.997529e+11,3.938015e+11,4.102483e+11,4.333287e+11,4.391487e+11,4.220465e+11,3.979899e+11
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.23282,1.211971e+11,1.319836e+11,1.363391e+11,1.379752e+11,1.480473e+11,1.587068e+11,1.660073e+11,1.756357e+11,1.828368e+11,1.850308e+11
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.60047,6.857344e+08,6.746459e+08,7.200629e+08,7.049483e+08,6.812260e+08,6.783400e+08,6.877044e+08,6.991892e+08,7.037801e+08,7.139983e+08
Russian Federation,5,18534,18301,34266,12422,1.85,57,5.900000e+07,70.0,28.29719,1.572241e+11,1.680156e+11,1.822280e+11,1.693503e+11,1.679981e+11,1.697726e+11,1.708608e+11,1.768949e+11,1.821293e+11,1.889349e+11
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.94543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Germany,7,17027,16831,140566,27426,8.26,126,1.630000e+08,40.0,82.22487,1.122006e+10,1.259477e+10,1.384270e+10,1.321497e+10,1.270950e+10,1.296059e+10,1.370547e+10,1.281359e+10,1.277716e+10,1.121913e+10
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.96908,2.249653e+11,2.374346e+11,2.323003e+11,2.192023e+11,2.200761e+11,2.257719e+11,2.261131e+11,2.293562e+11,2.412813e+11,2.601287e+11
France,9,13153,12973,130632,28601,9.93,114,1.374000e+09,252.0,19.53657,2.497135e+10,2.605681e+10,2.844096e+10,2.981914e+10,3.217477e+10,3.669404e+10,4.010384e+10,4.303644e+10,4.475182e+10,4.648940e+10


In [128]:
# space for proffesor tests

## Problem 2

The _Problem 1_ joined three datasets then reduced this to just the top $15$ entries. When you joined the datasets, but before you reduced this to the top $15$ items, how many entries did you lose?

_This function should return a single number._

In [129]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [130]:
def answer_two():
   # Joined datasets
    joinded = ((get_scimago_df()).join(generate_energy_df(), how='inner')).join(get_GDP_df(), how='inner')

    num_joined = len(joinded.index.unique()) # Number of unique countries after merging

    total_unique = len(set(generate_energy_df().index.unique()) | set(get_GDP_df().index.unique()) | set(get_scimago_df().index.unique()))
    countries_lost = total_unique - num_joined
    
    return countries_lost
    

In [131]:
# example of test cases
assert type(answer_two()) == int, "Q2: You should return an int number!"
print(answer_two())

147


In [132]:
# space for proffesor tests

## Problem 3

What are the top $15$ countries for average GDP over the last $10$ years?

_This function should return a Series named `avg_gdp` with $15$ countries and their average GDP sorted in descending order._

In [133]:
def gdp_clean_last_10_years():
    df = get_GDP_df()[list(i for i in range(2005,2016,1))].fillna(1) #Get last 10 years for each countr and fill the NaN with 1
    return df

def answer_three():
    avg_gdp = gdp_clean_last_10_years()
    avg_gdp['avg'] = [np.mean(list(avg_gdp.loc[country])) for country in avg_gdp.index] #Calculate the average in the last 10 years
    avg_gdp = avg_gdp.sort_values(by='avg', ascending=False)
    avg_gdp = avg_gdp.drop(index='Not classified')
    return avg_gdp['avg'][:15]


In [134]:
# example of test cases
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"
answer_three()

Country
Vietnam                       6.627827e+13
Guatemala                     4.556594e+13
Puerto Rico                   4.267742e+13
Hong Kong                     2.145282e+13
Lithuania                     2.026265e+13
Mexico                        1.997118e+13
Honduras                      1.986426e+13
Ethiopia                      1.708566e+13
Nauru                         1.693133e+13
Early-demographic dividend    1.677988e+13
Tuvalu                        1.534826e+13
Ukraine                       1.527741e+13
Low income                    1.350428e+13
Egypt, Arab Rep.              1.269560e+13
Dominican Republic            8.608860e+12
Name: avg, dtype: float64

In [135]:
# space for proffesor tests

# Problem 4

By how much had the GDP changed over the $10$ year span for the country with the $6-th$ largest average GDP?

_This function should return a single number._

In [136]:
def answer_four():
    df = list(gdp_clean_last_10_years().loc[answer_three().index[5]])
    return df[9] - df[0]

In [137]:
# example of test cases
assert type(answer_four()) == float, "Q4: You should return a single number!"
print(f"The {answer_three().index[5]}'s GPD has change: {answer_four()} in the last ten years")

The Mexico's GPD has change: 9695844554047.5 in the last ten years


In [138]:
# space for proffesor tests


## Problem 5

What is the _mean energy supply per capita_?

_This function should return a single number._

In [139]:
def answer_five():
    return float((generate_energy_df()['Energy Supply per capita']).mean())

In [140]:
# example of test cases
assert type(answer_five()) == float, "Q5: You should return a single number!"
print(f'The mean energy supply per capita is: {answer_five()}')

The mean energy supply per capita is: 90.66666666666667


In [141]:
# space for proffesor tests

## Problem 6

What __country__ has the _maximum % Renewable_ and what is the _percentage_?

_This function should return a tuple with the name of the country and the percentage._

In [142]:
def answer_six():
    df = generate_energy_df().sort_values(by='% Renewable', ascending=False)
    return (df.index[0],df['% Renewable'].iloc[0])
answer_six()

('Bhutan', 100)

In [143]:
# example of test cases
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"
print(f'{answer_six()[0]} is the country who has the maximum percentage renewable whit {answer_six()[1]}%')

Bhutan is the country who has the maximum percentage renewable whit 100%


In [144]:
# space for proffesor tests

## Problem 7

Create a new column that is the _ratio of Self-Citations to Total Citations_. 
What is the _maximum_ value for this new column, and what _country_ has the _highest ratio_?

_This function should return a tuple with the name of the country and the ratio._

In [145]:
def answer_seven():
    df = (answer_one())
    df['Citations ratio'] = df['Self-citations']/df['Citations']
    return (df['Citations ratio'].idxmax(),df['Citations ratio'].max())

In [146]:
# example of test cases
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"
print(f'{answer_seven()[0]} is the country who has the highest ratio whit {answer_seven()[1]} of ratio')

China is the country who has the highest ratio whit 0.6893126179389422 of ratio


In [147]:
# space for proffesor tests

## Problem 8

Create a column that estimates the _population using Energy Supply_ and _Energy Supply per capita_. 
What is the _third_ most populous country according to this estimate?

_This function should return the name of the country._

In [148]:
def get_population(df):
    return df['Energy Supply']/df['Energy Supply per capita']

def answer_eight():
    df = answer_one()
    df['Population'] = get_population(df)
    return (df.sort_values(by='Population',ascending=False)).index[2]

In [149]:
# example of test cases
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"
print(f'{answer_eight()} is the country who has the third most populous using Energy Supply and Energy Supply')


United States is the country who has the third most populous using Energy Supply and Energy Supply


In [150]:
# space for proffesor tests

## Problem 9

Create a column that estimates the _number of citable documents per person_. 

What is the correlation between the _number of citable documents per capita_ and the _energy supply per capita_? Use the `.corr()` method, (Pearson's correlation).

_This function should return a single number._

In [151]:
def answer_nine():
    df = answer_one()
    df['Citable doc per person'] = df['Citable documents']/(get_population(df))
    return df['Citable doc per person'].corr(df['Energy Supply per capita'])

In [152]:
# example of test cases
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"
answer_nine()

-0.2211754960400686

In [153]:
# space for proffesor tests

## Problem 10

Create a new column with a $1$ if the _country's % Renewable_ value is at or above the median for all countries in the _top 15_, and a $0$ if the _country's % Renewable_ value is below the median.

_This function should return a series named `high_renew` whose index is the country name sorted in ascending order of rank._

In [154]:
def answer_ten():
    df = answer_one()
    df['high_renew'] = df['% Renewable'].apply(lambda country: 1 if country >= np.mean(df['% Renewable']) else 0)
    return df['high_renew'].sort_values(ascending=True)

In [155]:
# example of test cases
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"
print(answer_ten())

Country
China                 0
United States         0
Japan                 0
United Kingdom        0
India                 0
France                0
South Korea           0
Iran                  0
Australia             0
Russian Federation    1
Canada                1
Germany               1
Italy                 1
Spain                 1
Brazil                1
Name: high_renew, dtype: int64


In [156]:
# space for proffesor tests

## Problem 11

Use the following dictionary to group the _Countries by Continent_, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the _sum_, _mean_, and _std deviation_ for the estimated population of each country.

```python
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}
```

_This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`_

In [157]:
def continent_col_creator(col):
    ContinentDict  = {
        'China':'Asia', 
        'United States':'North America', 
        'Japan':'Asia', 
        'United Kingdom':'Europe', 
        'Russian Federation':'Europe', 
        'Canada':'North America', 
        'Germany':'Europe', 
        'India':'Asia',
        'France':'Europe', 
        'South Korea':'Asia', 
        'Italy':'Europe', 
        'Spain':'Europe', 
        'Iran':'Asia',
        'Australia':'Australia', 
        'Brazil':'South America'
    }
    return [ContinentDict.get(country, country) for country in col]

def answer_eleven():
   
    df = answer_one()

    df['Continent'] = continent_col_creator(df.index)
    df['Population'] = get_population(df)

    return df.groupby('Continent')['Population'].agg(['size', 'sum', 'mean', 'std'])

In [158]:
# example of test cases
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"
answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5,2.902147e+09,5.804293e+08,6.784204e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,1.805929e+08,3.009881e+07,2.978893e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


In [159]:
# space for proffesor tests

## Problem 12

Cut (check `.cut()` from Pandas) _% Renewable_ into $5$ bins, but in the same column. Group _Top15_ by the _Continent_, as well as these new _% Renewable_ bins. How many countries are in each of these groups? You could check `.size()` as aggregation in group by.

_This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable` (in string datatype). Do not include groups with no countries._

In [160]:
def answer_twelve():
    df = answer_one()
    df['% Renewable'] = pd.cut(df['% Renewable'], bins=5)
    df['Continent'] = continent_col_creator(df.index)
    return df.groupby(['Continent','% Renewable'], observed=False).agg('size')

In [161]:
# example of test cases
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 25, "Q12: Wrong result numbers!"
answer_twelve()

Continent      % Renewable     
Asia           (1.606, 17.795]     4
               (17.795, 33.902]    1
               (33.902, 50.01]     0
               (50.01, 66.117]     0
               (66.117, 82.225]    0
Australia      (1.606, 17.795]     1
               (17.795, 33.902]    0
               (33.902, 50.01]     0
               (50.01, 66.117]     0
               (66.117, 82.225]    0
Europe         (1.606, 17.795]     1
               (17.795, 33.902]    3
               (33.902, 50.01]     1
               (50.01, 66.117]     0
               (66.117, 82.225]    1
North America  (1.606, 17.795]     1
               (17.795, 33.902]    0
               (33.902, 50.01]     0
               (50.01, 66.117]     1
               (66.117, 82.225]    0
South America  (1.606, 17.795]     0
               (17.795, 33.902]    0
               (33.902, 50.01]     0
               (50.01, 66.117]     0
               (66.117, 82.225]    1
dtype: int64

In [162]:
# space for proffesor tests

## Problem 13

Convert the _Population Estimate_ series to a string with thousands separator (using _dots_). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12.345.678,90

_This function should return a series `pop_est` whose index is the country name and whose values are the population estimate string_

In [163]:
def answer_thirteen():
    pop_est = pd.Series([(f'{number:,.10f}') for number in get_population(answer_one())])
    pop_est = pop_est.apply(lambda number: (number.replace(',', ';').replace('.', ',').replace(';', '.')))
    return (pop_est)


In [164]:
# example of test cases
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"
(answer_thirteen())

0     1.367.645.161,2903225422
1       317.615.384,6153846383
2       127.409.395,9731543660
3        63.870.967,7419354841
4           842.857,1428571428
5        35.239.864,8648648635
6         4.075.000,0000000000
7     1.276.730.769,2307691574
8         5.452.380,9523809524
9        53.285.714,2857142836
10       59.908.256,8807339445
11       46.443.396,2264150977
12       77.075.630,2521008402
13       23.316.017,3160173148
14      205.915.254,2372881472
dtype: object

In [165]:
# space for proffesor tests